<a href="https://colab.research.google.com/github/tara-shukla/jrw/blob/main/multi_mt5_model_jrw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install numba

from numba import cuda
device = cuda.get_current_device()
device.reset()


PYTORCH_CUDA_ALLOC_CONF=expandable_segments = True


In [ ]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 42.6 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [ ]:
!pip install datasets
!pip install transformers[torch]
! pip install -U accelerate
! pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 6.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
from datasets import load_dataset
import json
import os
import glob



#get train data file
!wget -q -O train.txt "https://docs.google.com/uc?export=download&id=1cQBB074e3-xffKNuDgUok5M_u7iuykYZ"

#get train corrected data file
!wget -q -O test.txt 'https://docs.google.com/uc?export=download&id=1P0IJRSSwxgAMPJP98bHnJNgzHqG-PfRl'

#get val data file
!wget -q -O val.txt 'https://docs.google.com/uc?export=download&id=18AvCV0nwvxSUA8U102enCqzdsLHsf04E'

#splitting text data and putting into json format
def formatdata(filename):
    with open((filename+".txt"), "r", encoding = 'utf-8') as file:
        data = []
        for line in file:
            error_sentence, correct_sentence = line.strip().split('\t')
            data.append({'error_sentence': error_sentence, 'correct_sentence': correct_sentence})

        indexed_data = []
        for index, pair in enumerate(data):
            indexed_pair = {'index': index, **pair}
            indexed_data.append(indexed_pair)

    json_data = json.dumps(indexed_data, ensure_ascii=False, indent=4)

    # Write JSON data
    with open((filename+".json"), "w", encoding = 'utf-8') as json_file:
        json_file.write(json_data)


formatdata("train")
formatdata("test")
formatdata("val")

#loading dataset
raw_datasets = load_dataset("json", data_files={'train': 'train.json', 'test': 'test.json', 'val':'val.json'})


raw_datasets = raw_datasets.filter(lambda x: x["correct_sentence"] is not None)
raw_datasets = raw_datasets.filter(lambda x: x["error_sentence"] is not None)
data_sample = raw_datasets["train"].shuffle(seed=63).select(range(1000))
# Peek at data
data_sample[:3]

raw_train_dataset = raw_datasets["train"]
#need to index like in tutorial? labels??
raw_train_dataset[0]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating val split: 0 examples [00:00, ? examples/s]

Filter:   0%|          | 0/19898 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4265 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4264 [00:00<?, ? examples/s]

Filter:   0%|          | 0/19898 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4265 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4264 [00:00<?, ? examples/s]

{'index': 0,
 'error_sentence': '사람들이 정년을 연장은 고령화 사회의 해결 방법이라고 생각했다.',
 'correct_sentence': '사람들이 정년 연장은 고령화 사회의 해결 방법이라고 생각했다.'}

In [ ]:

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/mt5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google/mt5-base")

tokenizer_config.json:   0%|          | 0.00/376 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/702 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
def preprocess_function(examples):
    inputs = examples["error_sentence"]
    targets = examples["correct_sentence"]
    model_inputs = tokenizer(
        inputs, text_target=targets, max_length=128, truncation=True
    )
    return model_inputs


tokenized_datasets = raw_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

tokenized_datasets

Map:   0%|          | 0/19898 [00:00<?, ? examples/s]

Map:   0%|          | 0/4265 [00:00<?, ? examples/s]

Map:   0%|          | 0/4264 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 19898
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 4265
    })
    val: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 4264
    })
})

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
batch = data_collator([tokenized_datasets["train"][i] for i in range(1, 3)])
batch.keys()
batch["decoder_input_ids"]

tensor([[     0,  21307,   1000,    381,   2479,  15486,    873,  11523,   8234,
          10169,   7025,  27383,    611,   6104,   1741,    259, 124607,  39514,
           3708,   8297,  11523,   8234,    747,  66866,  57049,   2317,    259,
            666,   2321,   1566,   3083,   2317,    259,  44781,    260],
        [     0,   1884,  18978,  15588,   5223,  94234,    259,   8297,  10878,
            869,   6104,  13958,  14984,   2321,   1566,   4214,    259,  61835,
          10174,  85561,    291,      1,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0]])

In [ ]:
from transformers import Seq2SeqTrainingArguments

args = Seq2SeqTrainingArguments(
    f"outputdir_me",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,
    predict_with_generate=True,
    push_to_hub=False,
    report_to="wandb",
)

In [ ]:
from nltk.translate.gleu_score import sentence_gleu as gleu_score
from sklearn.metrics import precision_recall_fscore_support
import evaluate
!pip install sacrebleu

gleu_metric = evaluate.load("google_bleu")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load("recall")
f1_metric = evaluate.load("f1")

bleu_metric = evaluate.load("sacrebleu")
metrics = [gleu_metric, precision_metric, recall_metric, f1_metric]


def compute_metrics(eval_preds):
    predictions, labels = eval_preds

    # In case the model returns more than the prediction logits
    if isinstance(predictions, tuple):
        predictions = predictions[0]

    predictions = np.where(predictions != -100, predictions, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]


    bleu = bleu_metric.compute(predictions=decoded_preds, references=decoded_labels)
    gleu = gleu_metric.compute(predictions=decoded_preds, references=decoded_labels)

    #for p, l in zip(decoded_preds, decoded_labels):
        #print(f"pred: {p}, label: {l}")

    #gleu = gleu_score(decoded_labels, decoded_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(decoded_labels, decoded_preds, average='weighted')
    #precision = precision_metric.compute(predictions=decoded_preds, references=decoded_labels)
    #recall = recall_metric.compute(predictions=decoded_preds, references=decoded_labels)
    #f1=f1_metric.compute(predictions=decoded_preds, references=decoded_labels)



    return {
      "bleu": bleu["score"],
      "gleu": gleu["google_bleu"],
      "f1": f1,
      "precision": precision,
      "recall": recall
      }


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 3.7 MB/s eta 0:00:00


In [ ]:
#make trainer
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["val"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


In [ ]:
import numpy as np
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

trainer.evaluate(max_length=128)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


{'eval_loss': 12.730679512023926,
 'eval_bleu': 0.26769796201367546,
 'eval_gleu': 0.018082718437328366,
 'eval_f1': 0.0,
 'eval_precision': 0.0,
 'eval_recall': 0.0,
 'eval_runtime': 85.6783,
 'eval_samples_per_second': 49.768,
 'eval_steps_per_second': 3.116}

In [ ]:
trainer.train()


Epoch,Training Loss,Validation Loss,Bleu,Gleu,F1,Precision,Recall
1,1.147600,0.754552,34.719484,0.378209,0.016886,0.016886,0.016886
2,0.786400,0.597724,36.805153,0.397277,0.045497,0.045497,0.045497
3,0.651500,0.557140,37.849599,0.406028,0.060976,0.060976,0.060976
4,0.626000,0.521668,38.611816,0.412226,0.071529,0.071529,0.071529
5,0.579300,0.503639,39.062153,0.416112,0.082317,0.082317,0.082317
6,0.562400,0.501806,39.198189,0.417194,0.084897,0.084897,0.084897
7,0.552200,0.487739,39.520425,0.419851,0.090760,0.090760,0.090760


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarnin

Epoch,Training Loss,Validation Loss,Bleu,Gleu,F1,Precision,Recall
1,1.147600,0.754552,34.719484,0.378209,0.016886,0.016886,0.016886
2,0.786400,0.597724,36.805153,0.397277,0.045497,0.045497,0.045497
3,0.651500,0.557140,37.849599,0.406028,0.060976,0.060976,0.060976
4,0.626000,0.521668,38.611816,0.412226,0.071529,0.071529,0.071529
5,0.579300,0.503639,39.062153,0.416112,0.082317,0.082317,0.082317
6,0.562400,0.501806,39.198189,0.417194,0.084897,0.084897,0.084897
7,0.552200,0.487739,39.520425,0.419851,0.090760,0.090760,0.090760
8,0.531800,0.480444,39.707160,0.421432,0.093496,0.093457,0.093574
9,0.525700,0.480472,39.878083,0.422936,0.094199,0.094160,0.094278
10,0.523600,0.477827,39.960861,0.423538,0.096310,0.096271,0.096388


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1141: UserWarning: Using the 

TrainOutput(global_step=12440, training_loss=0.7575966298388515, metrics={'train_runtime': 8203.6131, 'train_samples_per_second': 24.255, 'train_steps_per_second': 1.516, 'total_flos': 2.2874090644094976e+16, 'train_loss': 0.7575966298388515, 'epoch': 10.0})

In [ ]:
trainer.evaluate(max_length=128)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 0.4778271019458771,
 'eval_bleu': 61.9104111363851,
 'eval_gleu': 0.6427825227641702,
 'eval_f1': 0.1378205128205128,
 'eval_precision': 0.137781425891182,
 'eval_recall': 0.1378986866791745,
 'eval_runtime': 340.3749,
 'eval_samples_per_second': 12.527,
 'eval_steps_per_second': 0.784,
 'epoch': 10.0}

In [ ]:
next(model.parameters()).is_cuda

In [ ]:
trainer.evaluate(eval_dataset=tokenized_datasets["test"])

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'eval_loss': 0.4868788719177246,
 'eval_bleu': 39.306190956268196,
 'eval_gleu': 0.4177108323542448,
 'eval_f1': 0.08886283704572098,
 'eval_precision': 0.08886283704572098,
 'eval_recall': 0.08886283704572098,
 'eval_runtime': 147.3637,
 'eval_samples_per_second': 28.942,
 'eval_steps_per_second': 1.812,
 'epoch': 10.0}

In [ ]:

# Save the model to Google Drive
model_path = '/content/drive/My Drive/multi_mt5_model'
tokenizer_path = '/content/drive/My Drive/multi_mt5_model'


model.save_pretrained(model_path)
tokenizer.save_pretrained(tokenizer_path)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_path = '/content/drive/My Drive/multi_mt5_model'
tokenizer_path = '/content/drive/My Drive/multi_mt5_model'

tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)


In [ ]:
import torch
from nltk.translate.gleu_score import sentence_gleu as gleu_score
import nltk
nltk.download('punkt')
import torch

def finetuned_generate(text, model, tokenizer):
  raw_input_ids = tokenizer.encode(text)
  input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]
  corrected_ids = model.generate(torch.tensor([raw_input_ids]),
                                  max_length=128,
                                  eos_token_id=1, num_beams=4,
                                  early_stopping=True, repetition_penalty=2.0)

  output_text = tokenizer.decode(corrected_ids.squeeze().tolist(), skip_special_tokens=True)
  return output_text

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model = model.to('cpu')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
text = '한국어이어렵다고 생각하고 잇다.'
output_text=finetuned_generate(text, model, tokenizer)
print(output_text)

한국어가 어렵다고 생각하고 있다.


In [ ]:
output_file_path = "mt5_corrected_dataset.txt"

with open(output_file_path, "w", encoding="utf-8") as output_file:
    # Iterate through each tuple in the test dataset
    for sentences in raw_datasets["test"]:
        correct_sentence = sentences["correct_sentence"]
        error_sentence = sentences["error_sentence"]
        index = sentences["index"]
        corrected_sentence = finetuned_generate(error_sentence, model, tokenizer)

        # Write Corrected Sentence and Index to the Text File
        #output_file.write(f"Index: {index}\n")
        #output_file.write(f"Correct Sentence: {correct_sentence}\n")
        output_file.write(corrected_sentence)
        print(corrected_sentence)
        output_file.write("\n")

# Inform when all sentences are corrected and saved
print("All sentences corrected and saved to:", output_file_path)

본 이유로 TV에서 방영하는 영화의 방송 심의와 찬성한다는 입장이다.
지금부터 방콕을 소개한다.
페이스북은 유투복을 많이 사용을 했었습니다.
예를 들면 웰빙 음식으로 지즈 빵을 들 수 있다.
미국에서 저는 매운 음식을 못 먹었다.
요즘 같아선 선생님들도 포기하고 싶을 거예요.
하지만 한국에서 밥이 아주 비싸지만 많이 밥을 못 사요.
도운 아프라카와 추운 시비리아며 마음에 들으면 거기서 이사하는 것이 좋다.
나는 남자가 전업주부가 되는 것에 찬성한다.
그것은 인간의 우리가 지금까지 편하게 쉽게 생활할 수 있게 자연을 소중하게 생각하지 않고 우리 손으로 그렇게 만드는 것이다.
나무집을 짓을 때 나무를 오랫동안 살았으면 짓고 좋겠습니다.
그럼 다음 달을 봐요!
그래서 직장에서 그길 거라고 생각한다.
시월 이십육일에 제 생일이에요.
저는 다음에 음식을 유리하겠어요.
부모님께서는 고등학교에서 교사를 하시는데 항상 학생들을 생각하고 노력을 하시는 모습을 존중하고 있다.
토요일 저녁에는 이태원에서 맥주를 마시면서 삼겹살을 먹을 거예요.
한국어 영화가 아주 재미있습니다.
모두가 차를 타서 생활을 하다가는 공기가 나빠지거나 여름은 더 더워질 듯한다.
그것은 창의적 광고로 상품을 판매율을 높일 수 있냐고 생각했습니다.
일반인 중에도 마찬가지고 타인의 사생활을 보호하지 않고 공개되면 수많은 사건이 일어날 위험도 있다.
누가 몸 관리도 할 수 있는데 마음을 먹어야 할 수 있다.
나는 러시아에서 실고 싶지 않는데 한국에서 살면 좋겠다.
커피뿐만 아니라 더울 때 아이스크림을 먹으러 가자고 말하는 사람도 많이 있었다
하지만 대인과계가 원만하다보면 좀 공사 부분이 확실하지 않다는 줄을 때가 있습니다.
그리고 언제나 목표를 하나 가지는 것도 중요하다.
마을에서 어려운 이야기를 하고 의견이 왔다 갔다 해서 결론을 내릴 수 없을 때 점치는 것 같은 초인간적인 힘을 빌리는 경우가 많다고 한다.
그래서 이 학교 폭력 문제가 어떻게 발생하는지 소개한다.
그리고 쿄토에 가면 녹차를 먹어야 한다고 해서